# Accelerating Dask with Numba

In this notebook, we'll explore how you can accelerate parallel Dask DataFrame workflows by using `numba` to precompile code ahead of time. We've already seen how we can use `numba` in concert with `mpi4py` to accelerate parallel simulation programs earlier in the class. Here, we'll focus on using `numba` in a common analytical workflow -- that of applying some function to a column (or several columns) in your DataFrame using the `apply` method and creating a new, derived column for further study.

For this demonstration, we'll be working with a small sample of [AirBnB's listing data](http://insideairbnb.com/get-the-data.html), a large dataset that contains information on AirBnBs from around the world on a month-by-month basis. The methods described in this notebook are fully scalable (and can handle the full archive of AirBnB data if you wanted to), though, if you increase the number of workers (and memory) in your Dask cluster.

To begin, let's load in our packages and request resources to start up our Dask cluster (note that this notebook is meant to be run on the Midway Cluster):

In [12]:
import dask
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
import dask.dataframe as dd
from numba.pycc import CC
import numpy as np
import time

# Compose SLURM script
cluster = SLURMCluster(queue='broadwl', cores=4, memory='2GB', 
                       processes=4, walltime='00:15:00', interface='ib0',
                       job_extra=['--account=macs30123']
                      )

# Request resources
cluster.scale(jobs=1)

/home/jclindaniel/.local/lib/python3.7/site-packages/distributed/node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36873 instead
  expected, actual


In [25]:
! squeue -u jclindaniel

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          11272726   broadwl dask-wor jclindan  R       0:15      1 midway2-0012


In [26]:
client = Client(cluster)
client

Client Scheduler: tcp://172.25.220.71:42907 Dashboard: http://172.25.220.71:36873/status,Cluster Workers: 4 Cores: 4 Memory: 1.86 GiB


Then, we can load in our AirBnB data (included in this directory) and see what it looks like (note that this data is from three cities: Chicago, Boston, and San Francisco, compiled by AirBnB in April 2021):

In [27]:
df = dd.read_csv('listings*.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3781,HARBORSIDE-Walk to subway,4804,Frank,NaN,East Boston,42.36413,-71.02991,Entire home/apt,125,32,19,2021-02-26,0.27,1,106
1,6695,$99 Special!! Home Away! Condo,8229,Terry,NaN,Roxbury,42.32802,-71.09387,Entire home/apt,169,29,115,2019-11-02,0.81,4,40
2,10813,"Back Bay Apt-blocks to subway, Newbury St, The...",38997,Michelle,NaN,Back Bay,42.35061,-71.08787,Entire home/apt,96,29,5,2020-12-02,0.08,11,307
3,10986,North End (Waterfront area) CLOSE TO MGH & SU...,38997,Michelle,NaN,North End,42.36377,-71.05206,Entire home/apt,96,29,2,2016-05-23,0.03,11,293
4,13247,Back Bay studio apartment,51637,Susan,NaN,Back Bay,42.35164,-71.08752,Entire home/apt,75,91,0,NaN,NaN,2,0


You'll notice that two of the columns in the DataFrame are "latitude" and "longitude" -- spatial coordinates corresponding to AirBnB locations. Let's say that we're interested in creating a derived column from these coordinates, measuring how far each AirBnB is from the MACSS building at the University of Chicago (so that we can compute some further summary statistics about this column).

To measure this distance, we can write a Python function to calculate the distance between two sets of (longitude, latitude) coordinates using [great-circle distance](https://en.wikipedia.org/wiki/Great-circle_distance). We'll write another version of this function that uses `numba` to compile this function ahead of time. 

Finally, we can write an additional function to make use of these distance formulas and assess the distance of any coordinates from the MACSS building:

In [28]:
def distance(lon1, lat1, lon2, lat2):
    '''                                                                         
    Calculate the circle distance between two points                            
    on the earth (specified in decimal degrees)                                 
    '''
    # convert decimal degrees to radians                                        
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula                                                         
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    # 6367 km is the radius of the Earth                                        
    km = 6367 * c
    m = km * 1000
    return m

# Use Numba to compile this same function in a module named `aot`
cc = CC('aot')
@cc.export('distance', 'f8(f8,f8,f8,f8)')
def distance_numba(lon1, lat1, lon2, lat2):
    '''                                                                         
    Calculate the circle distance between two points                            
    on the earth (specified in decimal degrees)
    
    (Numba-accelerated version)
    '''
    # convert decimal degrees to radians                                        
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula                                                         
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    # 6367 km is the radius of the Earth                                        
    km = 6367 * c
    m = km * 1000
    return m
cc.compile()

# Import aot and incorporate both distance functions into single function that
# we'll apply to our dataframe
import aot
def distance_from_macss(lon, lat, numba=False):
    '''
    Compute distance to MACSS building (1155 E. 60th Street, Chicago, IL)
    from a given coordinate (longitude, latitude). Can accelerate with
    Numba if specify `numba=True` when calling function.
    '''
    macss_lon, macss_lat = -87.5970978, 41.7856443
    if numba:
        return aot.distance(lon, lat, macss_lon, macss_lat)
    return distance(lon, lat, macss_lon, macss_lat)

Then, we can "apply" this `distance_from_macss` function to our DataFrame, which will run our function in parallel on the different DataFrame partitions spread across our Dask workers (using the `map_partitions` method). We'll also produce some summary statistics using the `describe` method to get a sense of how our data is shaped. 

Note that we use both a plain-Python version of our code as well as our `numba`-accelerated one (setting `numba=True`) to see if we observe a performance boost by using our compiled distance function:

In [29]:
print("Dask alone:")
%timeit summary = df.map_partitions(lambda d: d.apply(lambda x: distance_from_macss(x.longitude, x.latitude), axis=1)) \
                    .describe()\
                    .compute()

print("Dask + Numba:")
%timeit summary = df.map_partitions(lambda d: d.apply(lambda x: distance_from_macss(x.longitude, x.latitude, numba=True), axis=1)) \
                    .describe() \
                    .compute()

Dask alone:
715 ms ± 112 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Dask + Numba:
487 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can see that we do in fact see a performance boost by using Numba to compile our code in addition to parallelizing it with Dask. This allows us to boost our performance and, for larger data sizes (and more computationally intensive function applications), this could make a major difference in our run time.

Then, with the time savings in computation, we have more time to analyze our data! For instance, we can see that the closest AirBnB to the MACSS building is only 150 meters away:

In [30]:
summary = df.map_partitions(lambda d: d.apply(lambda x: distance_from_macss(x.longitude, x.latitude, numba=True), axis=1)) \
                    .describe() \
                    .compute()
summary

count    1.621400e+04
mean     1.501024e+06
std      1.335917e+06
min      1.492782e+02
25%      1.427786e+04
50%      1.362680e+06
75%      2.985235e+06
max      2.995024e+06
dtype: float64

And, using more complex Pandas-like queries, we can find those nearby AirBnBs that match other relevant criteria (such as being located in the Hyde Park neighborhood in Chicago and having more than one review associated with them:

In [31]:
df['distance_from_macss'] = df.map_partitions(lambda d: d.apply(lambda x: distance_from_macss(x.longitude,
                                                                                              x.latitude,
                                                                                              numba=True), axis=1))

df[(df.distance_from_macss < 800) & (df.number_of_reviews > 1) & (df.neighbourhood == 'Hyde Park')].compute()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,distance_from_macss
283,3984383,Private room in house on Professor's Row at U ...,13339125,Greg,NaN,Hyde Park,41.78934,-87.58946,Private room,59,1,47,2019-09-22,0.59,2,0,754.425550
996,13857415,Really nice private room and bathroom with par...,20538405,Baila,NaN,Hyde Park,41.79046,-87.59743,Private room,77,2,162,2021-04-21,2.79,2,319,535.852608
1134,15250951,King suite - 2-bedroom suite with free parking,20538405,Baila,NaN,Hyde Park,41.79205,-87.59566,Private room,90,2,167,2021-04-13,3.03,2,237,721.733829
3347,34597480,"Huge, Bright One Bedroom on the Park",23006345,Madeline,NaN,Hyde Park,41.78900,-87.58865,Entire home/apt,219,1,2,2019-12-22,0.09,1,303,793.098505
